In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math
from sklearn.utils import shuffle

def load_data(filename):
    df = pd.read_csv(filename,sep=',')
    filename = filename.split('.')[0]
    print(filename)
    df.fillna(0, inplace=True) 
    df.loc[:, 'U'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
    df.loc[:, 'N'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
    new_flag_df = df['flags'].to_frame()
    new_flag_df = new_flag_df['flags'].str.get_dummies(' ')
    """
    1. Pro
    2. Packets
    3. Bytes
    4. Flags
    5. Durat
    6. Records
    7. svc_port - combination of sPort and dPort

    v = np.minimum(df['sPort'], df['dPort'])
    df['svc_port'] = np.where(v <= 1024, v, df['dPort'])
    """
    df['svc_port'] = df.apply(lambda x : min(x['sPort'],x['dPort']) if min(x['sPort'],x['dPort']) <= 1024 else x['dPort'], axis=1)
    
    if 'Records' in df:
        df = df.loc[:,('pro', 'packets', 'bytes', 'svc_port', 'durat','U','N', 'Records')]
        mawii_labels = pd.get_dummies(df['Records'],prefix='',prefix_sep='')
        new_df = df.drop(['Records'], axis=1)
    else:
        df = df.loc[:,('pro', 'packets', 'bytes', 'svc_port', 'durat','U','N', 'status')]
        mawii_labels = pd.get_dummies(df['status'],prefix='',prefix_sep='')
        new_df = df.drop(['status'], axis=1)
        
    mawii_features = pd.concat([new_df, new_flag_df], axis=1)
    total_df = pd.concat([mawii_features, mawii_labels], axis=1)
    shuffle_df = shuffle(total_df)
    file_name_labels = shuffle_df[['anomaly', 'normal']]
    file_name_features = shuffle_df.drop(file_name_labels, axis = 1)
    file_name_labels = file_name_labels[:int(len(file_name_labels) * 0.1)].as_matrix()
    file_name_features = file_name_features[:int(len(file_name_features) * 0.1)].as_matrix()
    file_name_features.dump(filename+"features.pkl")
    file_name_labels.dump(filename+"labels.pkl")
    print(file_name_features.shape)
    print(file_name_labels.shape)

In [36]:
load_data("20170827_mawilab_flow_000.csv")
load_data("20170827_mawilab_flow_001.csv")
load_data("20170827_mawilab_flow_002.csv")
load_data("20170827_mawilab_flow_003.csv")
load_data("20170827_mawilab_flow_004.csv")

20170827_mawilab_flow_000
(38361, 16)
(38361, 2)
20170827_mawilab_flow_001
(40780, 16)
(40780, 2)
20170827_mawilab_flow_002
(47265, 16)
(47265, 2)
20170827_mawilab_flow_003
(42398, 16)
(42398, 2)
20170827_mawilab_flow_004
(42599, 16)
(42599, 2)
